In [42]:
import sys
from pathlib import Path
import pandas as pd

# Notebook cell
%load_ext autoreload
%autoreload 2

# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))

# Verify path
print(f"Python will look in these locations:\n{sys.path}")


# --- Execute the processor ---
import utils
from config import date_str, DOWNLOAD_DIR, DEST_DIR

path_data = f'..\data\{date_str}_df_finviz_merged.parquet'
# path_corr = '..\data\df_corr_emv_matrix.parquet'
# path_cov = '..\data\df_cov_emv_matrix.parquet'
# path_output = f'..\picks\{date_str}_portf.txt'


print(utils.__file__)  # Should point to your src/utils.py
print(f'path_date: {path_data}')
# print(f'path_corr: {path_corr}')
# print(f'path_cov: {path_cov}')  
# print(f'path_output: {path_output}')
# print((f'date_str: {date_str}'))



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Python will look in these locations:
['C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\python311.zip', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\DLLs', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\Lib', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv', '', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\Pythonwin', 'c:\\Users\\ping\\Files_win10\\python\\py311\\stocks\\src', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\setuptools\\_vendor', 'c:\\Users\\ping\\Files_win10\\python\\py311\\stocks\\src', 'c:\\Users\\ping\\Files_w

In [43]:
import pandas as pd
# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
# pd.set_option('display.width', None)        # Let the display adjust to the window
# pd.set_option('display.max_colwidth', None) # Show full content of each cell
pd.set_option('display.max_rows', 100)
# pd.set_option('display.width', 120)
pd.set_option('display.float_format', '{:.4f}'.format)

In [44]:
import pandas as pd

def get_column_values_above_threshold(df, column_name='Avg Volume, M', threshold=0.75):
  """
  Analyzes the number and percentage of values in a DataFrame column that are above a specified threshold,
  and returns the filtered DataFrame.

  Args:
    df (pd.DataFrame): The input DataFrame.
    column_name (str): The name of the column to analyze. Defaults to 'Avg Volume, M'.
    threshold (float): The threshold value to compare against. Defaults to 1.00.

  Returns:
    pd.DataFrame: A DataFrame containing only the rows where the specified column's value is above the threshold.
  """
  
  count_before = len(df)
  above_threshold_df = df[df[column_name] > threshold]
  count_after = len(above_threshold_df)
  percentage = (count_after / len(df)) * 100

  print(f"count_before: {count_before}")
  print(f"count_after above threshold ({threshold}): {count_after}")
  print(f"Percentage above threshold ({threshold}): {percentage:.2f}%")

  return above_threshold_df


In [45]:
df_data = pd.read_parquet(path_data)

# liquidity filter, Avg Volume, M > 0.75M
df_data = get_column_values_above_threshold(df_data, column_name='Avg Volume, M', threshold=0.75)

# Drop specified columns with NaNs in df_data
df_data = df_data.drop(['All-Time High %', 'All-Time Low %', 'Dividend %'], axis=1)

# df_corr = pd.read_parquet(path_corr)
# df_cov = pd.read_parquet(path_cov)

# print(f'\ndf_cov.shape: {df_cov.shape}')
# display(df_cov.head())

# print(f'\ndf_corr.shape: {df_corr.shape}')
# display(df_corr.head())

print(f'\ndf_data.shape: {df_data.shape}')
display(df_data.head())
display((df_data.describe()))

count_before: 1538
count_after above threshold (0.75): 1163
Percentage above threshold (0.75): 75.62%

df_data.shape: (1163, 52)


,Company,Info,"MktCap AUM, M",Beta,RSI,Perf YTD %,Perf 3D %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,ATR,ATR/Price %,Volatility W %,Volatility M %,"Volume, M","Avg Volume, M",Rel Volume,Price,Change %,Sharpe 3d,Sortino 3d,Omega 3d,Sharpe 5d,Sortino 5d,Omega 5d,Sharpe 10d,Sortino 10d,Omega 10d,Sharpe 15d,Sortino 15d,Omega 15d,Sharpe 30d,Sortino 30d,Omega 30d,Sharpe 60d,Sortino 60d,Omega 60d,Sharpe 120d,Sortino 120d,Omega 120d,Sharpe 250d,Sortino 250d,Omega 250d
Ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AAPL,Apple Inc,"Technology, Consumer Electronics",2829860.0000,1.2800,25.1800,-24.7700,-15.5966,-13.5500,-19.9500,-22.7500,-16.9300,11.0400,-13.0100,-17.7500,-17.7400,-24.6500,-6.4000,-27.5700,14.8100,7.7200,4.0981,2.8700,2.7400,125.2546,53.4000,2.3500,188.3800,-7.2900,-95.0264,-15.7649,0.0000,-12.3849,-10.6367,0.0458,-6.6782,-6.6183,0.2503,-4.0381,-4.2295,0.4339,-5.3298,-5.4058,0.3591,-2.8086,-3.1374,0.5957,-1.3539,-1.5717,0.7762,0.4219,0.5637,1.0798
MSFT,Microsoft Corporation,"Technology, Software - Infrastructure",2675040.0000,0.9800,29.8100,-14.6300,-5.8479,-5.0100,-9.3400,-14.0300,-13.7300,-14.4200,-6.1600,-9.9900,-14.5000,-19.7500,-2.0200,-23.1700,-2.0200,9.4700,2.6317,2.5900,2.1000,48.6759,23.2700,2.0900,359.8400,-3.5600,-55.9690,-15.5646,0.0000,-6.9221,-7.7386,0.3002,-8.7696,-8.4783,0.2176,-5.4273,-5.7978,0.3985,-3.8311,-4.5377,0.5329,-2.5879,-3.1940,0.6311,-1.2700,-1.5673,0.7945,-0.7949,-1.0000,0.8724
NVDA,NVIDIA Corp,"Technology, Semiconductors",2301160.0000,2.1200,28.4500,-29.7700,-14.3804,-14.0100,-14.7100,-31.8100,-20.6500,6.0100,-16.5200,-22.2500,-25.7800,-36.6900,-7.1800,-38.4100,24.7400,6.0700,6.4362,4.5700,4.1500,527.5631,282.6600,1.8700,94.3100,-7.3600,-379.9017,-15.8676,0.0000,-10.6906,-9.8567,0.1215,-12.7153,-10.3649,0.0699,-7.7014,-7.4762,0.2492,-3.9790,-4.5100,0.5245,-2.1868,-2.5932,0.6873,-1.1436,-1.4172,0.8227,0.3606,0.4953,1.0617
AMZN,Amazon.com Inc,"Consumer Cyclical, Internet Retail",1812210.0000,1.3100,27.1900,-22.0600,-11.0163,-11.2700,-14.8000,-22.3500,-7.4500,-6.2600,-12.0600,-19.4600,-14.5900,-29.4900,-3.3500,-29.4900,12.7900,8.0100,4.6842,4.1900,3.1700,121.0205,41.5100,2.9200,171.0000,-4.1500,-30.6206,-14.9046,0.0000,-7.9740,-8.1611,0.2256,-8.4096,-7.9730,0.2027,-4.5575,-4.9039,0.4319,-4.4169,-4.7602,0.4695,-3.1662,-3.5835,0.5830,-0.6056,-0.7806,0.9034,-0.2455,-0.3226,0.9594
GOOG,Alphabet Inc,"Communication Services, Internet Content & Inf...",1788580.0000,1.0200,28.4600,-22.4200,-7.0116,-5.3300,-15.1900,-22.5000,-11.7000,-5.5200,-9.5400,-16.6900,-16.0300,-29.2100,-2.9200,-29.2100,-0.3100,5.0900,3.4452,3.1700,2.7100,39.4299,20.2200,1.9500,147.7400,-3.2000,-111.9152,-15.7953,0.0000,-8.2350,-8.5910,0.2338,-9.8588,-9.1066,0.1998,-5.5202,-5.9638,0.4206,-5.0140,-5.4770,0.4710,-3.4762,-3.9417,0.5668,-0.6620,-0.8781,0.8966,-0.1646,-0.2252,0.9721


,"MktCap AUM, M",Beta,RSI,Perf YTD %,Perf 3D %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,ATR,ATR/Price %,Volatility W %,Volatility M %,"Volume, M","Avg Volume, M",Rel Volume,Price,Change %,Sharpe 3d,Sortino 3d,Omega 3d,Sharpe 5d,Sortino 5d,Omega 5d,Sharpe 10d,Sortino 10d,Omega 10d,Sharpe 15d,Sortino 15d,Omega 15d,Sharpe 30d,Sortino 30d,Omega 30d,Sharpe 60d,Sortino 60d,Omega 60d,Sharpe 120d,Sortino 120d,Omega 120d,Sharpe 250d,Sortino 250d,Omega 250d
count,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000
mean,56746.2339,0.9958,33.5226,-8.8494,-9.0176,-8.5273,-9.6933,-8.9058,-9.0122,0.5537,-8.8107,-11.2545,-9.6523,-20.1059,0.5322,-25.0482,19.1920,3.5723,3.8855,3.1335,2.7759,13.1161,6.1416,2.1574,93.5898,-5.4726,-87.0122,28.3257,3.7771,-7.9127,-0.4466,1.1891,-5.8527,-4.6024,0.5331,-3.9131,-3.6843,0.5702,-2.4188,-2.5129,0.7076,-0.9023,-0.8633,0.8939,-0.6602,-0.7383,0.9036,0.0321,0.1386,1.0187
std,182742.0692,0.5713,11.0735,15.7237,5.9753,6.3558,9.1261,15.7189,18.1144,40.0802,6.7260,10.0066,13.6969,12.7941,8.1754,15.2516,45.9282,4.6849,2.3708,1.8305,1.5426,31.3313,13.7146,1.0670,105.9745,3.4185,760.8041,237.3052,21.0948,5.8974,99.1276,12.4649,4.7207,8.6047,1.0285,3.2747,4.3966,0.5474,2.3701,2.9397,0.3524,1.8719,2.5240,0.3228,1.1214,1.4933,0.1932,0.8704,1.2937,0.1678
min,3130.0000,-0.8600,11.4800,-71.1600,-45.2665,-45.5400,-54.6000,-68.4500,-75.0300,-83.8400,-50.7200,-61.6400,-73.8700,-75.0100,-23.4200,-95.1700,-23.4200,0.0200,0.0359,0.0100,0.0100,0.7612,0.7514,0.7000,2.1900,-23.4900,-5019.1620,-15.8745,0.0000,-70.4985,-15.5810,0.0000,-22.5982,-13.4373,0.0024,-14.1119,-11.0707,0.0452,-7.5506,-7.3686,0.1616,-5.7615,-6.1009,0.3055,-3.7874,-4.2304,0.4881,-2.5290,-2.9402,0.6301
25%,8590.0000,0.6450,25.1150,-18.7650,-12.3511,-12.2000,-15.3600,-18.6500,-18.6250,-17.1200,-12.9300,-17.4900,-18.2150,-27.9700,-5.1900,-34.5300,0.9100,0.8700,2.4926,1.9350,1.8800,2.7840,1.4800,1.5000,31.8400,-7.2850,-66.2639,-15.6515,0.0000,-10.9986,-10.0093,0.0789,-8.8162,-8.2329,0.1325,-5.8223,-5.9830,0.2887,-4.2766,-4.6407,0.4515,-2.3524,-2.7337,0.6503,-1.5052,-1.8280,0.7578,-0.5412,-0.7010,0.9062
50%,17480.0000,0.9700,30.6300,-8.7400,-9.0871,-8.7000,-9.8700,-8.9400,-10.3000,-2.4900,-8.8700,-11.0200,-9.3300,-18.5200,-1.4900,-22.8000,8.0500,2.1000,3.3829,2.8600,2.5700,5.2865,2.5900,1.9200,60.0800,-5.6800,-29.7342,-14.8515,0.0000,-9.1032,-8.9271,0.1515,-7.0220,-6.9452,0.2260,-4.5110,-4.8190,0.4042,-2.8370,-3.2706,0.5962,-1.1696,-1.5006,0.8093,-0.7554,-0.9940,0.8721,-0.0160,-0.0205,0.9973
75%,45455.0000,1.2350,39.2550,0.8300,-5.1462,-4.4050,-3.2250,0.8750,-0.1300,9.8700,-4.0100,-3.8200,-0.4150,-10.6600,4.2950,-13.8400,24.7400,4.5850,4.9117,4.0350,3.4850,11.9303,5.4250,2.5200,110.2000,-3.7250,-13.6019,-12.2437,0.0000,-6.1790,-6.8637,0.3143,-3.5238,-4.0726,0.4937,-2.3436,-2.7269,0.6378,-1.0163,-1.2868,0.8392,0.2955,0.3966,1.0529,0.1304,0.1917,1.0239,0.5692,0.8442,1.1055
max,2829860.0000,4.3300,89.7800,64.3800,22.2222,36.2800,39.8100,68.2300,97.4100,765.0900,22.0800,27.2900,52.5900,2.3500,68.4400,0.0500,908.5400,41.5000,25.0859,16.7600,14.3200,527.5631,282.6600,13.4600,916.4800,11.3300,22514.7926,1413.9041,126.9606,24.6254,2294.5944,290.0917,11.8933,129.7560,16.3847,8.2187,42.5984,6.8846,5.0630,9.9701,2.4378,4.1452,10.5661,2.3871,2.4905,5.6159,1.8470,3.7603,7.5436,1.8156


In [46]:
import pandas as pd
import os
import datetime

# --- Configuration ---
STATS_HISTORY_FILE = 'market_daily_summary_stats_history.csv' # Where to store the history

def append_daily_stats(df_daily_data, date_str, filename=STATS_HISTORY_FILE):
    """
    Calculates descriptive stats for df_daily_data, flattens them,
    and appends/updates them as a row in a historical CSV file.

    Args:
        df_daily_data (pd.DataFrame): DataFrame containing the raw data for ALL stocks for ONE specific day.
        date_str (str): The date for this data in 'YYYY-MM-DD' format.
        filename (str): The path to the CSV file storing the historical summary stats.
    """
    print(f"Processing stats for date: {date_str}")

    # 1. Calculate descriptive statistics for the input day's data
    try:
        daily_stats_raw = df_daily_data.describe()
        print("Successfully calculated describe() stats.")
    except Exception as e:
        print(f"Error calculating describe() stats: {e}")
        return # Stop if calculation fails

    # 2. Flatten the describe() output into a single row DataFrame
    try:
        # Unstack turns the stats (mean, std, etc.) into columns alongside the original metrics
        stats_flat = daily_stats_raw.unstack().to_frame().T
        # Create clearer column names like 'Metric_Statistic' (e.g., 'RSI_mean', 'SMA50 %_50%')
        stats_flat.columns = ['_'.join(col).strip() for col in stats_flat.columns.values]
        # Add the date to this row
        stats_flat['Date'] = pd.to_datetime(date_str)
        # Set the Date as the index for this single row
        stats_flat = stats_flat.set_index('Date')
        print("Successfully flattened stats into a single row.")
        # print(stats_flat) # Optional: print the flattened stats for verification

    except Exception as e:
        print(f"Error flattening the stats DataFrame: {e}")
        return # Stop if flattening fails

    # 3. Load existing history, append/update, and save
    try:
        df_summary_history = pd.DataFrame() # Initialize empty
        if os.path.exists(filename):
            print(f"Loading existing history from: {filename}")
            df_summary_history = pd.read_csv(filename, index_col='Date', parse_dates=True)

        # Check if the date already exists - update if it does, append if it doesn't
        current_date = pd.to_datetime(date_str)
        if current_date in df_summary_history.index:
            print(f"Updating existing entry for {date_str}")
            # Update existing row(s) - combines columns, replacing old values for this date
            df_summary_history.update(stats_flat)
            # Alternative: df_summary_history.loc[current_date] = stats_flat.iloc[0] # Direct overwrite
        else:
            print(f"Appending new entry for {date_str}")
            df_summary_history = pd.concat([df_summary_history, stats_flat])

        # Ensure the history is sorted by date
        df_summary_history = df_summary_history.sort_index()

        # Save the updated history back to CSV
        df_summary_history.to_csv(filename, index=True) # index=True saves the Date index
        print(f"Successfully saved updated stats history to: {filename}")

    except Exception as e:
        print(f"Error loading/saving the stats history file '{filename}': {e}")



In [47]:
# Simulate processing for a couple of days
print("-" * 20, "Day 1", "-" * 20)
append_daily_stats(df_data, date_str)

-------------------- Day 1 --------------------
Processing stats for date: 2025-04-04
Successfully calculated describe() stats.
Successfully flattened stats into a single row.
Loading existing history from: market_daily_summary_stats_history.csv
Appending new entry for 2025-04-04
Successfully saved updated stats history to: market_daily_summary_stats_history.csv


In [48]:
print(f'date_str: {date_str}')
df_data.describe()

date_str: 2025-04-04


,"MktCap AUM, M",Beta,RSI,Perf YTD %,Perf 3D %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,ATR,ATR/Price %,Volatility W %,Volatility M %,"Volume, M","Avg Volume, M",Rel Volume,Price,Change %,Sharpe 3d,Sortino 3d,Omega 3d,Sharpe 5d,Sortino 5d,Omega 5d,Sharpe 10d,Sortino 10d,Omega 10d,Sharpe 15d,Sortino 15d,Omega 15d,Sharpe 30d,Sortino 30d,Omega 30d,Sharpe 60d,Sortino 60d,Omega 60d,Sharpe 120d,Sortino 120d,Omega 120d,Sharpe 250d,Sortino 250d,Omega 250d
count,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000,1163.0000
mean,56746.2339,0.9958,33.5226,-8.8494,-9.0176,-8.5273,-9.6933,-8.9058,-9.0122,0.5537,-8.8107,-11.2545,-9.6523,-20.1059,0.5322,-25.0482,19.1920,3.5723,3.8855,3.1335,2.7759,13.1161,6.1416,2.1574,93.5898,-5.4726,-87.0122,28.3257,3.7771,-7.9127,-0.4466,1.1891,-5.8527,-4.6024,0.5331,-3.9131,-3.6843,0.5702,-2.4188,-2.5129,0.7076,-0.9023,-0.8633,0.8939,-0.6602,-0.7383,0.9036,0.0321,0.1386,1.0187
std,182742.0692,0.5713,11.0735,15.7237,5.9753,6.3558,9.1261,15.7189,18.1144,40.0802,6.7260,10.0066,13.6969,12.7941,8.1754,15.2516,45.9282,4.6849,2.3708,1.8305,1.5426,31.3313,13.7146,1.0670,105.9745,3.4185,760.8041,237.3052,21.0948,5.8974,99.1276,12.4649,4.7207,8.6047,1.0285,3.2747,4.3966,0.5474,2.3701,2.9397,0.3524,1.8719,2.5240,0.3228,1.1214,1.4933,0.1932,0.8704,1.2937,0.1678
min,3130.0000,-0.8600,11.4800,-71.1600,-45.2665,-45.5400,-54.6000,-68.4500,-75.0300,-83.8400,-50.7200,-61.6400,-73.8700,-75.0100,-23.4200,-95.1700,-23.4200,0.0200,0.0359,0.0100,0.0100,0.7612,0.7514,0.7000,2.1900,-23.4900,-5019.1620,-15.8745,0.0000,-70.4985,-15.5810,0.0000,-22.5982,-13.4373,0.0024,-14.1119,-11.0707,0.0452,-7.5506,-7.3686,0.1616,-5.7615,-6.1009,0.3055,-3.7874,-4.2304,0.4881,-2.5290,-2.9402,0.6301
25%,8590.0000,0.6450,25.1150,-18.7650,-12.3511,-12.2000,-15.3600,-18.6500,-18.6250,-17.1200,-12.9300,-17.4900,-18.2150,-27.9700,-5.1900,-34.5300,0.9100,0.8700,2.4926,1.9350,1.8800,2.7840,1.4800,1.5000,31.8400,-7.2850,-66.2639,-15.6515,0.0000,-10.9986,-10.0093,0.0789,-8.8162,-8.2329,0.1325,-5.8223,-5.9830,0.2887,-4.2766,-4.6407,0.4515,-2.3524,-2.7337,0.6503,-1.5052,-1.8280,0.7578,-0.5412,-0.7010,0.9062
50%,17480.0000,0.9700,30.6300,-8.7400,-9.0871,-8.7000,-9.8700,-8.9400,-10.3000,-2.4900,-8.8700,-11.0200,-9.3300,-18.5200,-1.4900,-22.8000,8.0500,2.1000,3.3829,2.8600,2.5700,5.2865,2.5900,1.9200,60.0800,-5.6800,-29.7342,-14.8515,0.0000,-9.1032,-8.9271,0.1515,-7.0220,-6.9452,0.2260,-4.5110,-4.8190,0.4042,-2.8370,-3.2706,0.5962,-1.1696,-1.5006,0.8093,-0.7554,-0.9940,0.8721,-0.0160,-0.0205,0.9973
75%,45455.0000,1.2350,39.2550,0.8300,-5.1462,-4.4050,-3.2250,0.8750,-0.1300,9.8700,-4.0100,-3.8200,-0.4150,-10.6600,4.2950,-13.8400,24.7400,4.5850,4.9117,4.0350,3.4850,11.9303,5.4250,2.5200,110.2000,-3.7250,-13.6019,-12.2437,0.0000,-6.1790,-6.8637,0.3143,-3.5238,-4.0726,0.4937,-2.3436,-2.7269,0.6378,-1.0163,-1.2868,0.8392,0.2955,0.3966,1.0529,0.1304,0.1917,1.0239,0.5692,0.8442,1.1055
max,2829860.0000,4.3300,89.7800,64.3800,22.2222,36.2800,39.8100,68.2300,97.4100,765.0900,22.0800,27.2900,52.5900,2.3500,68.4400,0.0500,908.5400,41.5000,25.0859,16.7600,14.3200,527.5631,282.6600,13.4600,916.4800,11.3300,22514.7926,1413.9041,126.9606,24.6254,2294.5944,290.0917,11.8933,129.7560,16.3847,8.2187,42.5984,6.8846,5.0630,9.9701,2.4378,4.1452,10.5661,2.3871,2.4905,5.6159,1.8470,3.7603,7.5436,1.8156


In [49]:
print(f'date_str: {date_str}')
df_data.head()

date_str: 2025-04-04


,Company,Info,"MktCap AUM, M",Beta,RSI,Perf YTD %,Perf 3D %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,ATR,ATR/Price %,Volatility W %,Volatility M %,"Volume, M","Avg Volume, M",Rel Volume,Price,Change %,Sharpe 3d,Sortino 3d,Omega 3d,Sharpe 5d,Sortino 5d,Omega 5d,Sharpe 10d,Sortino 10d,Omega 10d,Sharpe 15d,Sortino 15d,Omega 15d,Sharpe 30d,Sortino 30d,Omega 30d,Sharpe 60d,Sortino 60d,Omega 60d,Sharpe 120d,Sortino 120d,Omega 120d,Sharpe 250d,Sortino 250d,Omega 250d
Ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AAPL,Apple Inc,"Technology, Consumer Electronics",2829860.0000,1.2800,25.1800,-24.7700,-15.5966,-13.5500,-19.9500,-22.7500,-16.9300,11.0400,-13.0100,-17.7500,-17.7400,-24.6500,-6.4000,-27.5700,14.8100,7.7200,4.0981,2.8700,2.7400,125.2546,53.4000,2.3500,188.3800,-7.2900,-95.0264,-15.7649,0.0000,-12.3849,-10.6367,0.0458,-6.6782,-6.6183,0.2503,-4.0381,-4.2295,0.4339,-5.3298,-5.4058,0.3591,-2.8086,-3.1374,0.5957,-1.3539,-1.5717,0.7762,0.4219,0.5637,1.0798
MSFT,Microsoft Corporation,"Technology, Software - Infrastructure",2675040.0000,0.9800,29.8100,-14.6300,-5.8479,-5.0100,-9.3400,-14.0300,-13.7300,-14.4200,-6.1600,-9.9900,-14.5000,-19.7500,-2.0200,-23.1700,-2.0200,9.4700,2.6317,2.5900,2.1000,48.6759,23.2700,2.0900,359.8400,-3.5600,-55.9690,-15.5646,0.0000,-6.9221,-7.7386,0.3002,-8.7696,-8.4783,0.2176,-5.4273,-5.7978,0.3985,-3.8311,-4.5377,0.5329,-2.5879,-3.1940,0.6311,-1.2700,-1.5673,0.7945,-0.7949,-1.0000,0.8724
NVDA,NVIDIA Corp,"Technology, Semiconductors",2301160.0000,2.1200,28.4500,-29.7700,-14.3804,-14.0100,-14.7100,-31.8100,-20.6500,6.0100,-16.5200,-22.2500,-25.7800,-36.6900,-7.1800,-38.4100,24.7400,6.0700,6.4362,4.5700,4.1500,527.5631,282.6600,1.8700,94.3100,-7.3600,-379.9017,-15.8676,0.0000,-10.6906,-9.8567,0.1215,-12.7153,-10.3649,0.0699,-7.7014,-7.4762,0.2492,-3.9790,-4.5100,0.5245,-2.1868,-2.5932,0.6873,-1.1436,-1.4172,0.8227,0.3606,0.4953,1.0617
AMZN,Amazon.com Inc,"Consumer Cyclical, Internet Retail",1812210.0000,1.3100,27.1900,-22.0600,-11.0163,-11.2700,-14.8000,-22.3500,-7.4500,-6.2600,-12.0600,-19.4600,-14.5900,-29.4900,-3.3500,-29.4900,12.7900,8.0100,4.6842,4.1900,3.1700,121.0205,41.5100,2.9200,171.0000,-4.1500,-30.6206,-14.9046,0.0000,-7.9740,-8.1611,0.2256,-8.4096,-7.9730,0.2027,-4.5575,-4.9039,0.4319,-4.4169,-4.7602,0.4695,-3.1662,-3.5835,0.5830,-0.6056,-0.7806,0.9034,-0.2455,-0.3226,0.9594
GOOG,Alphabet Inc,"Communication Services, Internet Content & Inf...",1788580.0000,1.0200,28.4600,-22.4200,-7.0116,-5.3300,-15.1900,-22.5000,-11.7000,-5.5200,-9.5400,-16.6900,-16.0300,-29.2100,-2.9200,-29.2100,-0.3100,5.0900,3.4452,3.1700,2.7100,39.4299,20.2200,1.9500,147.7400,-3.2000,-111.9152,-15.7953,0.0000,-8.2350,-8.5910,0.2338,-9.8588,-9.1066,0.1998,-5.5202,-5.9638,0.4206,-5.0140,-5.4770,0.4710,-3.4762,-3.9417,0.5668,-0.6620,-0.8781,0.8966,-0.1646,-0.2252,0.9721
